<a href="https://colab.research.google.com/github/lis-r-barreto/Data-Engineering/blob/main/07_Practice_Optimizing_DataFrames_and_Processing_in_Chunks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataquest Guided Project: Optimizing Dataframes and Processing in Chunks

We'll be working with financial lending data from Lending Club, a marketplace for personal loans that matches borrowers with investors.

In this project, it is assumed that  that we only have 10 megabytes of memory available throughout this project, so you can practice the concepts of process large CSV as chunk and reduce memeory usage of chunks by analysis the data in chunk and set data types of columns accordingly.

The official Dataquest reference solution is in GitHub repo [here](https://github.com/dataquestio/solutions/blob/master/Mission165Solutions.ipynb).

My solution is slighthly different from the 'reference' solution.  As I try to keep the column data type (and thier size) the same across all the chunks and categorize category column using distinct values across all chunks instead of only distinct value in a chunk. 

In [ ]:
import pandas as pd
import numpy as np

import pprint as pp

pd.options.display.max_columns = 99

## Explore the first 5 rows  ... there is date columns

Date columns is found, these columns should read as datetiem at later stage

In [ ]:
loan=pd.read_csv('loans_2007.csv',nrows=5)

loan

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


## Assume memory is limited ... control the chunk size
It is assume only 10M of memroy is available, so we need to control the memory of each chunk below 5M.  It turn out chunksize of 3000 is within our memory limit.

In [ ]:
loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000)

for loanChunk in loanChunkIter:
    print(loanChunk.memory_usage(deep=True).sum()/(1024*1024))


4.649013519287109
4.6447601318359375
4.646517753601074
4.647870063781738
4.6440629959106445
4.6459455490112305
4.644536972045898
4.646905899047852
4.645031929016113
4.645082473754883
4.657794952392578
4.6566619873046875
4.663469314575195
4.896910667419434
0.8808088302612305


## Explore the no. of numeric and string (object) columns
The initial steps is to check no. of numeric and string (object) columns in the dataset

In [ ]:
loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000)


loanChunkNumTxtColCounts=pd.DataFrame(columns=['Num Counts','Text Counts'])
for loanChunk in loanChunkIter:
    
    numColCount=len(loanChunk.select_dtypes(include='number').columns)
    txtColCount=len(loanChunk.select_dtypes(include='object').columns)
    loanChunkNumTxtColCounts.loc[len(loanChunkNumTxtColCounts)]=[numColCount,txtColCount]


In [ ]:
loanChunkNumTxtColCounts

,Num Counts,Text Counts
0,31,21
1,31,21
2,31,21
3,31,21
4,31,21
5,31,21
6,31,21
7,31,21
8,31,21
9,31,21


## Why number of numeric and text colum is not the same across all chunks ?

From the above data exploration/analysis, it is found the number of numeric column and object column is not the same for the last 2 chunks is different from all other chunks, need to find out which column caused the problem.

In [ ]:
loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000)


loanChunkTypes=pd.DataFrame(columns=loan.columns)
loanChunkUniqueValues=pd.DataFrame(columns=loan.columns)
for loanChunk in loanChunkIter:
    loanChunkTypes.loc[len(loanChunkTypes)]=loanChunk.dtypes
    


In [ ]:
loanChunkTypes

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
1,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
2,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
3,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
4,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
5,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
6,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
7,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
8,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float6

## What's wrong with the Id column ?

From the above data exploration/analysis, it is found the Id column in last 2 chunk is not number (int64), need further explore the reason.

In [ ]:
loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000)

for chunkNum,loanChunk in enumerate(loanChunkIter):
    if loanChunk['id'].dtypes=='object':
        print('Chunk {} has problem for this row=>'.format(chunkNum))
        strIDRow=loanChunk[loanChunk['id'].str.isdigit()==False].index
        print(loanChunk.loc[list(strIDRow)])
    else:
        print('Chunk {} no problem'.format(chunkNum))


Chunk 0 no problem
Chunk 1 no problem
Chunk 2 no problem
Chunk 3 no problem
Chunk 4 no problem
Chunk 5 no problem
Chunk 6 no problem
Chunk 7 no problem
Chunk 8 no problem
Chunk 9 no problem
Chunk 10 no problem
Chunk 11 no problem
Chunk 12 no problem
Chunk 13 has problem for this row=>
                                             id  member_id  loan_amnt  \
39786  Loans that do not meet the credit policy        NaN        NaN   

       funded_amnt  funded_amnt_inv term int_rate  installment grade  \
39786          NaN              NaN  NaN      NaN          NaN   NaN   

      sub_grade emp_title emp_length home_ownership  annual_inc  \
39786       NaN       NaN        NaN            NaN         NaN   

      verification_status issue_d loan_status pymnt_plan purpose title  \
39786                 NaN     NaN         NaN        NaN     NaN   NaN   

      zip_code addr_state  dti  delinq_2yrs earliest_cr_line  inq_last_6mths  \
39786      NaN        NaN  NaN          NaN              N

## 3 Rows are not containing loan information

From the above analysis
-  1 row in Chunk 13 is subheader, all other columns are NaN
-  2 rows in Chunk 14 are subtotals, which are not individual loan record

These rows needed to be removed.  Add A small function to remove "non-loan" rows in Chunk


In [ ]:
def removeProblematicRow(loanChunk):
    if loanChunk['id'].dtypes=='object':
        strIDRow=loanChunk[loanChunk['id'].str.isdigit()==False].index
        loanChunk.drop(list(strIDRow),axis=0,inplace=True)
        loanChunk['id']=loanChunk['id'].astype('int64')
    return loanChunk

In [ ]:
loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000)

loanChunkNumTxtColCounts=pd.DataFrame(columns=['Num Counts','Text Counts'])
loanChunkTypes=pd.DataFrame(columns=loan.columns)
for loanChunk in loanChunkIter:
    loanChunk=removeProblematicRow(loanChunk)
    
    numColCount=len(loanChunk.select_dtypes(include='number').columns)
    txtColCount=len(loanChunk.select_dtypes(include='object').columns)
    loanChunkNumTxtColCounts.loc[len(loanChunkNumTxtColCounts)]=[numColCount,txtColCount]
    
    loanChunkTypes.loc[len(loanChunkTypes)]=loanChunk.dtypes
    

In [ ]:
loanChunkNumTxtColCounts

,Num Counts,Text Counts
0,31,21
1,31,21
2,31,21
3,31,21
4,31,21
5,31,21
6,31,21
7,31,21
8,31,21
9,31,21


In [ ]:
loanChunkTypes

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
1,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
2,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
3,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
4,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
5,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
6,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
7,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object,float64,float64,object,float64,float64,float64,float64,float64
8,int64,float64,float64,float64,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,object,object,object,float64,float6

## No More Inconsistencce Across Chunks After "non-loan" rows removal
From the above, it can show that is no more inconsistence of column data types across all churnks after problematic rows are removed from concenred churnks


## Any missing values ?

The below code count the number of missing value for each columns for each chunk and then sum up missing value count for all chunks.

In [ ]:
loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000)


loanChunkMissings=pd.DataFrame(columns=loan.columns)
for loanChunk in loanChunkIter:
    loanChunk=removeProblematicRow(loanChunk)
    
    loanChunkMissing=[loanChunk[col].isnull().sum() for col in loanChunk.columns]
    loanChunkMissings.loc[len(loanChunkMissings)]=loanChunkMissing

loanChunkMissing=loanChunkMissings.sum(axis=0)

loanChunkMissing

id                               0.0
member_id                        0.0
loan_amnt                        0.0
funded_amnt                      0.0
funded_amnt_inv                  0.0
term                             0.0
int_rate                         0.0
installment                      0.0
grade                            0.0
sub_grade                        0.0
emp_title                     2626.0
emp_length                    1112.0
home_ownership                   0.0
annual_inc                       4.0
verification_status              0.0
issue_d                          0.0
loan_status                      0.0
pymnt_plan                       0.0
purpose                          0.0
title                           13.0
zip_code                         0.0
addr_state                       0.0
dti                              0.0
delinq_2yrs                     29.0
earliest_cr_line                29.0
inq_last_6mths                  29.0
open_acc                        29.0
p

## Which float columns has no missing value and could be candidates for conversion to the integer type?

Since we have checked data types for the same column in all chunk are the same afer "non-loan" rows are removed, the below code use the the column data type of last chunk from the previous step and missing value count to filter out float columns have no missing values. 

In [ ]:
colToInteger=loanChunk.columns[(loanChunk.dtypes=='float64') & (loanChunkMissing==0)]

print(colToInteger)

Index(['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'installment', 'dti', 'revol_bal', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'policy_code'],
      dtype='object')


## Check candidate of Integer columns are only integer value

Check across all chunk to see candidate integer columns only contain integer value across all chunk, if not remove the column from the candidate list.  The integer column list is saved for later use.

In [ ]:
loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000)

colIsFloat=set()
for loanChunk in loanChunkIter:
    loanChunk=removeProblematicRow(loanChunk)
    
    for col in loanChunk.columns:
        if col in colToInteger:
            allRowsInteger=all([x.is_integer() for x in loanChunk[col]])
            if allRowsInteger==False:
                colIsFloat.add(col)

colToInteger=[x for x in colToInteger if x not in colIsFloat]
colToInteger

['member_id', 'loan_amnt', 'funded_amnt', 'revol_bal', 'policy_code']

## Which string columns contain values that are less than 50% unique?

To answer the above we first create a list of dict to hold unique values of string (object) columns.

In [ ]:
loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000)

uniqueDicts=[{'col': col, 'non-numeric':False,'values':{}} for col in loan.columns]
totalNumberOfRows=0
for loanChunk in loanChunkIter:
    loanChunk=removeProblematicRow(loanChunk)
    totalNumberOfRows+=loanChunk.shape[0]

    for idx,col in enumerate(loanChunk):
        if loanChunk[col].dtypes=='object':
            uniqueDicts[idx]['non-numeric']=True
            for cell in loanChunk[col]:
                if cell in uniqueDicts[idx]['values']:
                    uniqueDicts[idx]['values'][cell]+=1
                else:
                    uniqueDicts[idx]['values'][cell]=1
               

## But wait ... not all string columns are strings ...
Explore the dict and found 7th and 30th column (int_rate, revol_util column) are holding percentage value, which should be of float type instead of string.  These column should be clean up first.

In [ ]:
pp.pprint(uniqueDicts[6])
pp.pprint(uniqueDicts[29])

{'col': 'int_rate',
 'non-numeric': True,
 'values': {'  5.42%': 573,
            '  5.79%': 410,
            '  5.99%': 347,
            '  6.00%': 19,
            '  6.03%': 447,
            '  6.17%': 252,
            '  6.39%': 58,
            '  6.54%': 307,
            '  6.62%': 396,
            '  6.76%': 168,
            '  6.91%': 310,
            '  6.92%': 204,
            '  6.99%': 336,
            '  7.05%': 23,
            '  7.12%': 9,
            '  7.14%': 342,
            '  7.29%': 397,
            '  7.37%': 32,
            '  7.40%': 72,
            '  7.42%': 7,
            '  7.43%': 33,
            '  7.49%': 656,
            '  7.51%': 787,
            '  7.66%': 292,
            '  7.68%': 94,
            '  7.74%': 154,
            '  7.75%': 27,
            '  7.88%': 742,
            '  7.90%': 582,
            '  7.91%': 14,
            '  8.00%': 198,
            '  8.07%': 26,
            '  8.32%': 49,
            '  8.38%': 26,
            '  8.49%':

## How about the term columns, change to integer ?
The term column only has two distinct values '36 months' and '60 months'.  Prefer to treat it as categorical rather than numeric. 

In [ ]:
pp.pprint(uniqueDicts[5])

{'col': 'term',
 'non-numeric': True,
 'values': {' 36 months': 31534, ' 60 months': 11001}}


## Recall date columns is identified from the first 5 rows
They should be read as date instead of string and not categorical. Put the column name in a list and pass the list to parse_date parmeter of read_csv

In [ ]:
dateCols=['issue_d','earliest_cr_line','last_pymnt_d','last_credit_pull_d']

## Let's clean the int_rate column and look for string columns contain values that are less than 50% unique again

Add a small function to replace % char in int_rate column and set the column type to float, then build the list of unique values dict again

In [ ]:
def cleanRateCol(loanChunk):
    loanChunk['int_rate']=loanChunk['int_rate'].str.replace('%','')
    loanChunk['int_rate']=loanChunk['int_rate'].astype('float')
    loanChunk['revol_util']=loanChunk['revol_util'].str.replace('%','')
    loanChunk['revol_util']=loanChunk['revol_util'].astype('float')


    
    return loanChunk

In [ ]:
loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000,parse_dates=dateCols)

totalNumberOfRows=0
uniqueDicts=[{'col': col, 'non-numeric':False,'values':{}} for col in loan.columns]

for loanChunk in loanChunkIter:
    loanChunk=removeProblematicRow(loanChunk)
    loanChunk=cleanRateCol(loanChunk)
    totalNumberOfRows+=loanChunk.shape[0]
    
    for idx,col in enumerate(loanChunk):
        if loanChunk[col].dtypes=='object':
            uniqueDicts[idx]['non-numeric']=True
            for cell in loanChunk[col]:
                if cell in uniqueDicts[idx]['values']:
                    uniqueDicts[idx]['values'][cell]+=1
                else:
                    uniqueDicts[idx]['values'][cell]=1
    

isCatCols=[False for x in uniqueDicts]

for idx,uniqueDict in enumerate(uniqueDicts):
    if uniqueDict['non-numeric']==True:
        noOfUniqueValues=len(uniqueDict['values'].keys())
        if noOfUniqueValues/totalNumberOfRows<0.5:
            isCatCols[idx]=True        

catCols=[y  for x,y in zip(isCatCols,loanChunk.columns) if x==True]
catCols

['term',
 'grade',
 'sub_grade',
 'emp_length',
 'home_ownership',
 'verification_status',
 'loan_status',
 'pymnt_plan',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'initial_list_status',
 'application_type']

## Column purpose, title are categorical ?  Probablly not  ...

For the initial exploration of the first 5 rows of the loan data set, puropse and titile look likie descriptive text columns.Descrpitive text columns are not common candidiates of 'categorical' as they are free text columns.  These two columns need further explore why unique values are taking up below 50%.

Print out the unique values of purpose column and value of title column that occur more than 10 times.  It is found that title has repetitive text of same meaning of different form (e.g 'wedding','wedding loan','Wedding Fund') and the purpose column already summarize the repetitive meaning of title.  So, the title column can be dropped and keep the purpose column as categorical columns.

In [ ]:
uniqueDicts[18]

{'col': 'purpose',
 'non-numeric': True,
 'values': {'car': 1615,
  'credit_card': 5477,
  'debt_consolidation': 19776,
  'educational': 422,
  'home_improvement': 3199,
  'house': 426,
  'major_purchase': 2311,
  'medical': 753,
  'moving': 629,
  'other': 4425,
  'renewable_energy': 106,
  'small_business': 1992,
  'vacation': 400,
  'wedding': 1004}}

In [ ]:
print(uniqueDicts[19]['col'])
for value in uniqueDicts[19]['values'].items():
    if value[1]>10:
        print(value)
    

title
(nan, 13)
('Dept Consolidation', 13)
('car', 30)
('HOME IMPROVEMENT', 15)
('My loan', 34)
('Business Expansion', 13)
('Freedom Loan', 21)
('Other Loan', 192)
('pool loan', 13)
('Pay off', 17)
('Medical Bills', 11)
('Boat Loan', 25)
('Consolidate', 141)
('Home Repairs', 11)
('CC Refinance', 27)
('Consolidating Credit Card Debt', 14)
('Pay off credit card', 11)
('Debit Consolidation', 20)
('moving', 15)
('Purchase', 12)
('Lending Club Loan', 33)
('Bill Consolidation', 48)
('loan1', 11)
('Debt loan', 13)
('Motorcycle loan', 16)
('truck', 11)
('Consolidate Bills', 12)
('Loan Consolidation', 37)
('consolidate debt', 25)
('debt_consolidation', 68)
('Consolidate Credit Cards', 16)
('business', 16)
('medical expenses', 12)
('Fresh Start', 19)
('Engagement Ring', 60)
('Debt Relief', 17)
('Credit Card Pay Off', 21)
('Pay off credit cards', 30)
('Pool loan', 12)
('Personal', 330)
('get out of debt', 21)
('Home Improvement Loan', 255)
('CONSOLIDATION', 14)
('Pay Off Credit Cards', 20)
('moto

## Rebulid the list of categorical columns and unique values dict of categorical columns again

So we write a small function to drop the titile column , rebulid the list of categorical columns and unique values dict of categorical columns agaian after dropping the title column



In [ ]:
def dropTitleCol(loanChunk):
    loanChunk.drop(columns=['title'],axis=1,inplace=True)
    
    return loanChunk

In [ ]:
loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000,parse_dates=dateCols)

totalNumberOfRows=0
uniqueDicts=[{'col': col, 'non-numeric':False,'values':{}} for col in loan.columns]

for loanChunk in loanChunkIter:
    loanChunk=removeProblematicRow(loanChunk)
    loanChunk=cleanRateCol(loanChunk)
    loanChunk=dropTitleCol(loanChunk)
    
    totalNumberOfRows+=loanChunk.shape[0]
    
    for idx,col in enumerate(loanChunk):
        if loanChunk[col].dtypes=='object':
            uniqueDicts[idx]['non-numeric']=True
            for cell in loanChunk[col]:
                if cell in uniqueDicts[idx]['values']:
                    uniqueDicts[idx]['values'][cell]+=1
                else:
                    uniqueDicts[idx]['values'][cell]=1

isCatCols=[False for x in uniqueDicts]

for idx,uniqueDict in enumerate(uniqueDicts):
    if uniqueDict['non-numeric']==True:
        noOfUniqueValues=len(uniqueDict['values'].keys())
        if noOfUniqueValues/totalNumberOfRows<0.5:
            isCatCols[idx]=True        

catCols=[y  for x,y in zip(isCatCols,loanChunk.columns) if x==True]
catCols    

['term',
 'grade',
 'sub_grade',
 'emp_length',
 'home_ownership',
 'verification_status',
 'loan_status',
 'pymnt_plan',
 'purpose',
 'zip_code',
 'addr_state',
 'initial_list_status',
 'application_type']

## For each identified category column, build a unique value list across all chunk

Write a small function to create a CategoricalDtype for each column, holding the unique values across all chunks of that column.

In [ ]:
from pandas.api.types import CategoricalDtype

def categorizeLoanChunk(loanChunk):
    for idx,col in enumerate(loanChunk.columns):
        if col in catCols:
            catValues=[value for value in uniqueDicts[idx]['values'].keys() if str(value)!='nan']
            cat_type = CategoricalDtype(categories=catValues)            
            loanChunk[col]=loanChunk[col].astype(cat_type)
    
    return loanChunk        
            

## A two pass startegy to categorized all category column
We want to use the same categoryization scheme for each cateogry column across all chunks.  To achieve this goal, a two pass startegy is emplyeed to categorize all category column across all chunks.

- First build the category column list and unqiue values dict across all chunks
- Then use the unique values dict to create a CategoryDtype for each category column to categorize the column.

To verify the result, list the datatypes of each chunks and sample check a category column of the last loanChunk.

In [ ]:
loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000,parse_dates=dateCols)

totalNumberOfRows=0
uniqueDicts=[{'col': col, 'non-numeric':False,'values':{}} for col in loan.columns]

for loanChunk in loanChunkIter:
    loanChunk=removeProblematicRow(loanChunk)
    loanChunk=cleanRateCol(loanChunk)
    loanChunk=dropTitleCol(loanChunk)
    
    totalNumberOfRows+=loanChunk.shape[0]
    
    for idx,col in enumerate(loanChunk):
        if loanChunk[col].dtypes=='object':
            uniqueDicts[idx]['non-numeric']=True
            for cell in loanChunk[col]:
                if cell in uniqueDicts[idx]['values']:
                    uniqueDicts[idx]['values'][cell]+=1
                else:
                    uniqueDicts[idx]['values'][cell]=1

loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000,parse_dates=dateCols)

loanChunkTypes=pd.DataFrame(columns=loanChunk.columns)

for loanChunk in loanChunkIter:
    loanChunk=removeProblematicRow(loanChunk)
    loanChunk=cleanRateCol(loanChunk)
    loanChunk=dropTitleCol(loanChunk)
    loanChunk=categorizeLoanChunk(loanChunk)

    loanChunkTypes.loc[len(loanChunkTypes)]=loanChunk.dtypes
    
loanChunkTypes

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,int64,float64,float64,float64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,datetime64[ns],float64,float64,float64,float64,float64,float64,category,float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],float64,datetime64[ns],float64,float64,category,float64,float64,float64,float64,float64
1,int64,float64,float64,float64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,datetime64[ns],float64,float64,float64,float64,float64,float64,category,float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],float64,datetime64[ns],float64,float64,category,float64,float64,float64,float64,float64
2,int64,float64,float64,float64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,datetime64[ns],float64,float64,float64,float64,float64,float64,category,float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],float64,datetime64[ns],float64,float64,category,float64,float64,float64,float64,float64
3,int64,float64,float64,float64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,datetime64[ns],float64,float64,float64,float64,float64,float64,category,float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],float64,datetime64[ns],float64,float64,category,float64,float64,float64,float64,float64
4,int64,float64,float64,float64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,datetime64[ns],float64,float64,float64,float64,float64,float64,category,float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],float64,datetime64[ns],float64,float64,category,float64,float64,float64,float64,float64
5,int64,float64,float64,float64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,datetime64[ns],float64,float64,float64,float64,float64,float64,category,float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],float64,datetime64[ns],float64,float64,category,float64,float64,float64,float64,float64
6,int64,float64,float64,float64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,datetime64[ns],float64,float64,float64,float64,float64,float64,category,float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],float64,datetime64[ns],float64,float64,category,float64,float64,float64,float64,float64
7,int64,float64,float64,float64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,date

In [ ]:
print('No of distinct addr_state values in last chunk => {}'.format(len(loanChunk['addr_state'].cat.codes.unique())))

print('No of category value used to categorized addr_state => {}'.format(len(loanChunk['addr_state'].cat.categories)))

No of distinct addr_state values in last chunk => 42
No of category value used to categorized addr_state => 50


## Process integer columns
After the categorical column is processed, it is time to process numeric columns.  Recall that we have already identified columns that can be converted to integer column and saved the integer column list in colToInteger.

A small function is created to covert the concerned column from float to integer.  The datatype of all columns across all chunks are then listed again to verify result.

In [ ]:
colToInteger

['member_id', 'loan_amnt', 'funded_amnt', 'revol_bal', 'policy_code']

In [ ]:
def convertColToInteger(loanChunk,colToInteger):
    for col in loanChunk.columns:
        if col in colToInteger:
            loanChunk[col]=loanChunk[col].astype('int64')
    return loanChunk        

In [ ]:
loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000,parse_dates=dateCols)

totalNumberOfRows=0
uniqueDicts=[{'col': col, 'non-numeric':False,'values':{}} for col in loan.columns]

for loanChunk in loanChunkIter:
    loanChunk=removeProblematicRow(loanChunk)
    loanChunk=cleanRateCol(loanChunk)
    loanChunk=dropTitleCol(loanChunk)
    
    totalNumberOfRows+=loanChunk.shape[0]
    
    for idx,col in enumerate(loanChunk):
        if loanChunk[col].dtypes=='object':
            uniqueDicts[idx]['non-numeric']=True
            for cell in loanChunk[col]:
                if cell in uniqueDicts[idx]['values']:
                    uniqueDicts[idx]['values'][cell]+=1
                else:
                    uniqueDicts[idx]['values'][cell]=1

loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000,parse_dates=dateCols)

loanChunkTypes=pd.DataFrame(columns=loanChunk.columns)

for loanChunk in loanChunkIter:
    loanChunk=removeProblematicRow(loanChunk)
    loanChunk=cleanRateCol(loanChunk)
    loanChunk=dropTitleCol(loanChunk)
    loanChunk=categorizeLoanChunk(loanChunk)
    loanChunk=convertColToInteger(loanChunk,colToInteger)
   
    loanChunkTypes.loc[len(loanChunkTypes)]=loanChunk.dtypes

loanChunkTypes

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,int64,int64,int64,int64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,datetime64[ns],float64,float64,float64,int64,float64,float64,category,float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],float64,datetime64[ns],float64,int64,category,float64,float64,float64,float64,float64
1,int64,int64,int64,int64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,datetime64[ns],float64,float64,float64,int64,float64,float64,category,float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],float64,datetime64[ns],float64,int64,category,float64,float64,float64,float64,float64
2,int64,int64,int64,int64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,datetime64[ns],float64,float64,float64,int64,float64,float64,category,float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],float64,datetime64[ns],float64,int64,category,float64,float64,float64,float64,float64
3,int64,int64,int64,int64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,datetime64[ns],float64,float64,float64,int64,float64,float64,category,float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],float64,datetime64[ns],float64,int64,category,float64,float64,float64,float64,float64
4,int64,int64,int64,int64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,datetime64[ns],float64,float64,float64,int64,float64,float64,category,float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],float64,datetime64[ns],float64,int64,category,float64,float64,float64,float64,float64
5,int64,int64,int64,int64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,datetime64[ns],float64,float64,float64,int64,float64,float64,category,float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],float64,datetime64[ns],float64,int64,category,float64,float64,float64,float64,float64
6,int64,int64,int64,int64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,datetime64[ns],float64,float64,float64,int64,float64,float64,category,float64,float64,float64,float64,float64,float64,float64,float64,float64,datetime64[ns],float64,datetime64[ns],float64,int64,category,float64,float64,float64,float64,float64
7,int64,int64,int64,int64,float64,category,float64,float64,category,category,object,category,category,float64,category,datetime64[ns],category,category,category,category,category,float64,float64,datetime64[ns],float64,float64,float64,int64,float64,float64,category,float64,fl

## Further reduce chunk size by using the apporpirate number of bits for integer or float type

64 bit is the default size used to represent integer and float in DataFrame.  The chunk size can be further reduce if we know the min and max value of numeric columns across all chunks and use the proper number of bits to represent integer and float value.

To determine the min and max of numeric value across all chunks, all chunks are read again to determine the min/max value and the result are stored in a dictionary as tuple.  

In [ ]:
loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000,parse_dates=dateCols)

totalNumberOfRows=0
uniqueDicts=[{'col': col, 'non-numeric':False,'values':{}} for col in loan.columns]

for loanChunk in loanChunkIter:
    loanChunk=removeProblematicRow(loanChunk)
    loanChunk=cleanRateCol(loanChunk)
    loanChunk=dropTitleCol(loanChunk)
    
    totalNumberOfRows+=loanChunk.shape[0]
    
    for idx,col in enumerate(loanChunk):
        if loanChunk[col].dtypes=='object':
            uniqueDicts[idx]['non-numeric']=True
            for cell in loanChunk[col]:
                if cell in uniqueDicts[idx]['values']:
                    uniqueDicts[idx]['values'][cell]+=1
                else:
                    uniqueDicts[idx]['values'][cell]=1

loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000,parse_dates=dateCols)

loanChunkTypes=pd.DataFrame(columns=loanChunk.columns)


numColMinMaxDict={}
for loanChunk in loanChunkIter:
    loanChunk=removeProblematicRow(loanChunk)
    loanChunk=cleanRateCol(loanChunk)
    loanChunk=dropTitleCol(loanChunk)
    loanChunk=categorizeLoanChunk(loanChunk)
    loanChunk=convertColToInteger(loanChunk,colToInteger)
   
    for numCol in loanChunk.select_dtypes(include='number'):
        if numCol not in numColMinMaxDict.keys():
            numColMinMaxDict[numCol]=(loanChunk[numCol].min(),loanChunk[numCol].max())
        else:
            colMin=loanChunk[numCol].min()
            colMax=loanChunk[numCol].max()
            if colMin<numColMinMaxDict[numCol][0]:
                numColMinMaxDict[numCol]=(colMin,numColMinMaxDict[numCol][1])
            if colMax>numColMinMaxDict[numCol][1]:
                numColMinMaxDict[numCol]=(numColMinMaxDict[numCol][0],colMax)


In [ ]:
numColMinMaxDict

{'acc_now_delinq': (0.0, 1.0),
 'annual_inc': (1896.0, 6000000.0),
 'chargeoff_within_12_mths': (0.0, 0.0),
 'collection_recovery_fee': (0.0, 7002.19),
 'collections_12_mths_ex_med': (0.0, 0.0),
 'delinq_2yrs': (0.0, 13.0),
 'delinq_amnt': (0.0, 6053.0),
 'dti': (0.0, 29.99),
 'funded_amnt': (500, 35000),
 'funded_amnt_inv': (0.0, 35000.0),
 'id': (54734, 1077501),
 'inq_last_6mths': (0.0, 33.0),
 'installment': (15.67, 1305.19),
 'int_rate': (5.42, 24.59),
 'last_pymnt_amnt': (0.0, 36115.2),
 'loan_amnt': (500, 35000),
 'member_id': (70473, 1314167),
 'open_acc': (1.0, 47.0),
 'out_prncp': (0.0, 5794.29),
 'out_prncp_inv': (0.0, 5794.29),
 'policy_code': (1, 1),
 'pub_rec': (0.0, 5.0),
 'pub_rec_bankruptcies': (0.0, 2.0),
 'recoveries': (0.0, 29623.35),
 'revol_bal': (0, 1207359),
 'revol_util': (0.0, 119.0),
 'tax_liens': (0.0, 1.0),
 'total_acc': (1.0, 90.0),
 'total_pymnt': (0.0, 58563.6799293133),
 'total_pymnt_inv': (0.0, 58563.68),
 'total_rec_int': (0.0, 23611.1),
 'total_rec_l

## The min/max dict shows there is no -ve value ...

The min/max dictionary shows that there is no -ve value in all numeric columns.  So unsigned integer can be used for integer values, and there is no need to check for -ve minimun value in determine the no. of bits to store numeric values.

Two procdure are created to size the integer (unsigned) columns and float columns using the np.iinfo() and np.finfo() to check the min and max value that can be represented with different numner of bits.  There two functions are then called to converted numeric columns using the apporpriate no. of bits that could hold the min/max of different numeric columns.

The final size of each chunk is then displayed and the data type of each column for all chunks are also displayed for verification.

In [ ]:
def sizeIntCol(loanChunk):
    for col in loanChunk.select_dtypes('integer'):
        if numColMinMaxDict[col][1]<np.iinfo(np.uint8).max:
            loanChunk[col]=loanChunk[col].astype('uint8')
        elif numColMinMaxDict[col][1]<np.iinfo(np.uint16).max:     
            loanChunk[col]=loanChunk[col].astype('uint16')
        elif numColMinMaxDict[col][1]<np.iinfo(np.uint32).max:     
            loanChunk[col]=loanChunk[col].astype('uint32')
        elif numColMinMaxDict[col][1]<np.iinfo(np.uint64).max:     
            loanChunk[col]=loanChunk[col].astype('uint64')
    
    return loanChunk 
        

In [ ]:
def sizeFloatCol(loanChunk):
    for col in loanChunk.select_dtypes('float'):
        if numColMinMaxDict[col][1]<np.finfo(np.float16).max:     
            loanChunk[col]=loanChunk[col].astype('float16')
        elif numColMinMaxDict[col][1]<np.finfo(np.float32).max:     
            loanChunk[col]=loanChunk[col].astype('float32')
        elif numColMinMaxDict[col][1]<np.finfo(np.float64).max:     
            loanChunk[col]=loanChunk[col].astype('float64')
    
    return loanChunk 
        

In [ ]:
loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000,parse_dates=dateCols)

totalNumberOfRows=0
uniqueDicts=[{'col': col, 'non-numeric':False,'values':{}} for col in loan.columns]

for loanChunk in loanChunkIter:
    loanChunk=removeProblematicRow(loanChunk)
    loanChunk=cleanRateCol(loanChunk)
    loanChunk=dropTitleCol(loanChunk)
    
    totalNumberOfRows+=loanChunk.shape[0]
    
    for idx,col in enumerate(loanChunk):
        if loanChunk[col].dtypes=='object':
            uniqueDicts[idx]['non-numeric']=True
            for cell in loanChunk[col]:
                if cell in uniqueDicts[idx]['values']:
                    uniqueDicts[idx]['values'][cell]+=1
                else:
                    uniqueDicts[idx]['values'][cell]=1

loanChunkIter = pd.read_csv("loans_2007.csv", chunksize=3000,parse_dates=dateCols)

loanChunkTypes=pd.DataFrame(columns=loanChunk.columns)

for loanChunk in loanChunkIter:
    loanChunk=removeProblematicRow(loanChunk)
    loanChunk=cleanRateCol(loanChunk)
    loanChunk=dropTitleCol(loanChunk)
    loanChunk=categorizeLoanChunk(loanChunk)
    loanChunk=convertColToInteger(loanChunk,colToInteger)
    loacnChunk=sizeIntCol(loanChunk)
    loacnChunk=sizeFloatCol(loanChunk)
   
   
    loanChunkTypes.loc[len(loanChunkTypes)]=loanChunk.dtypes
    print(loanChunk.memory_usage(deep=True).sum()/(1024*1024))
    
loanChunkTypes

0.6524324417114258
0.6490945816040039
0.6510372161865234
0.6513290405273438
0.6495151519775391
0.6511297225952148
0.649439811706543
0.6513757705688477
0.6512508392333984
0.6525535583496094
0.6515703201293945
0.6487932205200195
0.6552047729492188
0.6734914779663086
0.20389747619628906


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,uint32,uint32,uint16,uint16,float16,category,float16,float16,category,category,object,category,category,float32,category,datetime64[ns],category,category,category,category,category,float16,float16,datetime64[ns],float16,float16,float16,uint32,float16,float16,category,float16,float16,float16,float16,float16,float16,float16,float16,float16,datetime64[ns],float16,datetime64[ns],float16,uint8,category,float16,float16,float16,float16,float16
1,uint32,uint32,uint16,uint16,float16,category,float16,float16,category,category,object,category,category,float32,category,datetime64[ns],category,category,category,category,category,float16,float16,datetime64[ns],float16,float16,float16,uint32,float16,float16,category,float16,float16,float16,float16,float16,float16,float16,float16,float16,datetime64[ns],float16,datetime64[ns],float16,uint8,category,float16,float16,float16,float16,float16
2,uint32,uint32,uint16,uint16,float16,category,float16,float16,category,category,object,category,category,float32,category,datetime64[ns],category,category,category,category,category,float16,float16,datetime64[ns],float16,float16,float16,uint32,float16,float16,category,float16,float16,float16,float16,float16,float16,float16,float16,float16,datetime64[ns],float16,datetime64[ns],float16,uint8,category,float16,float16,float16,float16,float16
3,uint32,uint32,uint16,uint16,float16,category,float16,float16,category,category,object,category,category,float32,category,datetime64[ns],category,category,category,category,category,float16,float16,datetime64[ns],float16,float16,float16,uint32,float16,float16,category,float16,float16,float16,float16,float16,float16,float16,float16,float16,datetime64[ns],float16,datetime64[ns],float16,uint8,category,float16,float16,float16,float16,float16
4,uint32,uint32,uint16,uint16,float16,category,float16,float16,category,category,object,category,category,float32,category,datetime64[ns],category,category,category,category,category,float16,float16,datetime64[ns],float16,float16,float16,uint32,float16,float16,category,float16,float16,float16,float16,float16,float16,float16,float16,float16,datetime64[ns],float16,datetime64[ns],float16,uint8,category,float16,float16,float16,float16,float16
5,uint32,uint32,uint16,uint16,float16,category,float16,float16,category,category,object,category,category,float32,category,datetime64[ns],category,category,category,category,category,float16,float16,datetime64[ns],float16,float16,float16,uint32,float16,float16,category,float16,float16,float16,float16,float16,float16,float16,float16,float16,datetime64[ns],float16,datetime64[ns],float16,uint8,category,float16,float16,float16,float16,float16
6,uint32,uint32,uint16,uint16,float16,category,float16,float16,category,category,object,category,category,float32,category,datetime64[ns],category,category,category,category,category,float16,float16,datetime64[ns],float16,float16,float16,uint32,float16,float16,category,float16,float16,float16,float16,float16,float16,float16,float16,float16,datetime64[ns],float16,datetime64[ns],float16,uint8,category,float16,float16,float16,float16,float16
7,uint32,uint32,uint16,uint16,float16,category,float16,float16,category,category,object,category,category,float32,category,datetime64[ns],category,category,category,category,category,float16,float16,datetime64[ns],float16,float16,float16,ui

## Processing Result

We have effectively reduce the memory foot print of each chunk from 4.6M to 0.6M by
- read in date column as date type using date parse capability provided by pd.read_csv() function
- convert percentage value to float by removing the % character in the CSV
- categorize string columns for columns that the no. of unique value is less than half (50%) of total record in the loan CSV
- drop the free text column title
- convert numeric column to integer and float accordingly and store them in the apporpirate format in minimun no. of bits that could whole the min/max value

## Further processing ...

In exploring the data during the exerice, the below can further be processed ...
- remove 'months' from the terms column and categorize the column as numeric instead of text column
- similary remove 'years' in column emp_length and categorize the column as numeric instead opf text column  
- remove columns that are single valued across all chunk, as single valued column cannot be analyzed
